<a href="https://colab.research.google.com/github/mwingter/Calculo-Numerico/blob/master/Trabalho/Calculonumerico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Importações para rodar as bibliotecas necessárias --------------------------------------------


In [ ]:
import numpy as np
import timeit
import time

# **MÉTODO DE RESOLUÇÃO DE SISTEMAS LINEARES**

In [ ]:
n = 4 #dimensão do sistema
A = np.array([[1,1,0,3],[2,1,-1,1],[3,-1,-1,2],[-1,2,3,-1]]) #matrix do sistema linear
b = np.array([4,1,-3,4]) #vetor solução

- Método Direto

**I) Eliminação de Gauss --------------------** 



In [ ]:
def elimination(A, b, n):
  for i in range(n):
    pivot = A[i][i]
    for k in range(i+1,n):
      Mik = A[k][i] / pivot
      b[k] = b[k] - Mik * b[i]
      A[k] = A[k] - Mik * A[i] 
  
  return A, b

In [ ]:
def retro_substitution(A, b, n):
  x = np.zeros(n)
  for i in range(n-1, -1,-1):
    x[i] = b[i]
    sum = 0 
    j = n-1
    while (j>i):
      sum = sum + A[i][j] * x[j]
      j = j-1  
    x[i] = (x[i] - sum)/A[i][i]
  
  return x

In [ ]:
def gauss_elimination (A, b, n):
  C, blinha = elimination(A,b,n)
  x = retro_substitution(C,blinha,n)
  print("Resposta do sistema linear:", x)

gauss_elimination(A, b, n)

Resposta do sistema linear: [-3.92857143  6.85714286 13.        ]


- Métodos Iterativos

**II) Método de Gauss-Jacobi --------------------**


In [ ]:
n = 3 #dimensão do sistema
A = np.array([[6,3,1],[4,9,-3],[1,-1,3]]) #matrix do sistema linear
b = np.array([10,16,14]) #vetor solução
x0 = np.array([-1.8,5.5,7.3]) # chute inicial
x = np.vstack((x0,np.zeros(3)))
epsilon = 0.1 #erro
limit = 100 #limite de iteracao

In [ ]:
%%timeit
gauss_jacobi(A,b,x,n)

5.7
2.442857142857143
1.2214285714285715
0.0
The slowest run took 284.18 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.61 µs per loop


In [ ]:
def gauss_jacobi(A,b,x,n,e):
  k = 1
  while (np.max(np.abs(x[1]-x[0])) > e) and (k < limit):
    if (k!=1): x[0] = x[1]
    for i in range(n):
      sum = 0 
      for j in range(n):
        if j != i: 
          sum = sum + (A[i][j] * x[0][j])
      x[1][i] = ((b[i] - sum) / A[i][i])
    k = k + 1
    print(np.max(np.abs(x[1]-x[0])))
  return x[1]

**III) Método de Gauss-Seidel --------------------**

In [ ]:
n = 4 #dimensão do sistema
A = np.array([[2.0,-1.0,0,0],[1.0,2.0,-1.0,0],[0,-1.0,2.0,-1.0],[0,0,-1.0,2.0]]) #matriz
b = np.array([1,1,1,1]) #vetor b
xi = np.array([1.0,1.0,1.0,1.0]) #vetor aproximação inicial
x = np.array([1.0,1.0,1.0,1.0]) #vetor aproximação
e = 0.001 #erro

In [ ]:
def iteration(A, b, xi, x, n): ##iteração k, em que se calcula os valores do array x
    for i in range(n):
        sum1 = 0 
        sum2 = 0
        for j in range(0, i):
            sum1 = sum1 + A[i][j] * x[j]
        for j in range (i+1, n):
            sum2 = sum2 + A[i][j] * xi[j]
        x[i] = (1/A[i][i]) * (b[i] - sum1 - sum2)

In [ ]:
def stopCriteria(x, xi, n, e): #Critério de parada. Se retornar true, o critério foi satisfeito; se false, não foi.
    max = -99999
    for i in range(n):
        tot = abs(x[i] - xi[i])
        if(max < tot):
            max = tot
    if(max < e):
        return True
    else:
        return False

In [ ]:
def gauss_seidel(A, b, xi, x, e, n):
    k = 1 #contador de iterações
    while(k < 25): # limite máximo de iterações
        iteration(A, b, xi, x, n)
        if(stopCriteria(x, xi, n, e)):
            return x
        else:
            k = k + 1
            for i in range(n):
                xi[i] = x[i]

In [ ]:
#print("Resposta do método Gauss-Seidel:", gauss_seidel(A, b, xi, x, e, n))

In [ ]:
for i in range(3,300):
  e = 10e-5
  A,b = random_matrix_vectors(i)
  #metodo direto
  gauss_elimination(A,b,i)
  #metodos iterativos
  x = np.ones(i)
  gauss_jacobi(A,b,x,i,e)
  gauss_seidel(A,b,x,x,e,i)

In [ ]:
def random_matrix_vectors(n):
  A =  np.random.rand(n,n)
  b = np.random.rand(n)
  return A,b

In [ ]:
inicio = time.time()
random_matrix_vectors(100)
fim = time.time()

In [ ]:
fim - inicio

0.0008726119995117188

In [ ]:
A

array([[ 2., -1.,  0.,  0.],
       [ 1.,  2., -1.,  0.],
       [ 0., -1.,  2., -1.],
       [ 0.,  0., -1.,  2.]])

# **MÉTODO DOS MÍNIMOS QUADRADOS**

In [ ]:
f = np.array([0.459,0.828,1.006,1.150,1.354,1.261,1.157,0.834,0.511]) #vetor de valores f
M = np.array([[0.01,0.04,0.09,0.16,0.25,0.36,0.49,0.64,0.81],[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],[1,1,1,1,1,1,1,1,1]]) #matriz com os n vetores hk
n = 3 #dimensão da matriz simétrica A

In [ ]:
def min_quadrados(f, M, n):
    A = np.zeros((n,n)) # matriz n x n cheia de zeros
    b = np.zeros(n) #vetor b de tamanho n cheia de zeros
    for i in range(n):
        for j in range(i+1):
            A[i][j] = round(np.dot(M[i],M[j]), 3) # np.dot calcula o produto escalar entre dois vetores
            A[j][i] = round(np.dot(M[i],M[j]), 3)
    
    for i in range(n):
        b[i] = round(np.dot(f,M[i]), 3)

    print(A)
    print("\n", b)

    # utilizando o método de resolução de sistemas lineares  
    # para encontrar os coeficientes da solução
    return gauss_elimination(A, b, n)

In [ ]:
print(min_quadrados(f, M, n))

[[1.533 2.025 2.85 ]
 [2.025 2.85  4.5  ]
 [2.85  4.5   9.   ]]

 [2.619 4.344 8.56 ]
Resposta do sistema linear: [-5.10382514  5.2104918  -0.0379235 ]
None


# **MÉTODO DE RESOLUÇÃO DE SISTEMAS NÃO LINEARES**

**I) Método de Newton**

In [ ]:
n = 2 #numero de elementos em x
xi = np.array([-1, 6]) #vetor aproximação inicial
F = np.array([[1,1,-5],[1,1,-25]])
e = 0.1 #erro e

In [ ]:
def toJacobian(f, x, n):
    # IMPLEMENTAR
    return j

In [ ]:
def iteration_newton(n, xi):
    Jx = np.zeros((n,n)) #(A) matriz n x n cheia de zeros 
    Fx = np.zeros(n) #(b) vetor de tamanho n cheia de zeros 
    y = np.zeros(n) #(x)
    #encontrando o sistema Ax=b 
    for i in range(n): #Encontrando o 'b' (Fx)
        Fx[i] = F[i][n]
        for j in range(n):
            Fx[i] = Fx[i] + F[i][j] * (xi[j] ** (i+1))
    print(Fx)
    toJacobian(Fx, xi, n) #encontrando o 'A' (Jx)

    x = gaussElimination.gauss(A, b, n) #resolvendo o sistema linear A.x=b ou Jx.x=Fx

    x = x + y
    return x

In [ ]:
def newton(n, xi, e):
    k = 1 #contador de iterações
    while (k < 25):
        print("---ITERAÇÃO ", k)
        x = iteration_newton(n, xi)
        k = k + 1
        for i in range(n):
            xi = x

# **METODO DE AUTOVALORES E AUTOVETORES**

**I) Método das Potências**

In [ ]:
k = 1
A = np.array([[-4,14,0],[-5,13,0],[-1,0,2]]) #matriz
x = np.array([1.0,1.0,1.0]) #vetor aproximação inicial 



In [ ]:
z = np.dot(A,x.T)
p = np.argmax(z)
z = z/z[p]
z


array([1. , 0.8, 0.1])

In [ ]:

def power_method(A,x,epsilon,limit):
  for k in range(limit):
    x1 = np.dot(A,x)
    p = np.argmax(x1)
    lam = x1[p]
    x1 = x1/x1[p]
    diff = x - x1  
    if x1[p] == 0:
      return 0, x
    if (np.linalg.norm(diff) < epsilon):
      return lam, x1
    x = x1
  print("Rodou iteracoes", k)
  return lam, x

In [ ]:
lam, x = power_method(A,x,10e-9,100)
print("Autovalor:", lam)
print("Autovetor:", x)

Autovalor: 6.000000051089694
Autovetor: [ 1.          0.71428572 -0.24999999]
